## Imports

In [1]:
import pdfplumber
import pandas as pd
import numpy as np
from datetime import datetime

## Functions

In [2]:
def get_date(str_date):
    global year
    date = datetime.strptime(str_date+year, '%d-%b%Y')
    return date

def is_date(date):
    try:
        get_date(date)
        return True
    except ValueError:
        return False

def is_float(str_float):
    try:
        float(str_float)
        if str_float[-3] == ".":
            return True
        else:
            return False
    except ValueError:
        return False

In [3]:
def enter_row_into_statement_dict(date, details, amount, balance):
    global statement_dict
    
    statement_dict["Date"] += [date]
    statement_dict["Transaction Details"] += [details]
    statement_dict["Amount"] += [amount]
    statement_dict["Balance"] += [balance]
    
def clear_statement_dict():
    global statement_dict
    statement_dict["Date"] = []
    statement_dict["Transaction Details"] = []
    statement_dict["Amount"] = []
    statement_dict["Balance"] = []

In [4]:
def check_if_two_transactions(items):
    global balance
    global amount
    global details
    
    second_item_is_number = is_float(items[-2])
    
    if second_item_is_number:
        balance = items[-1]
        amount = items[-2]
        details = " ".join(items[1:-2])
    else:
        amount = items[-1]
        details = " ".join(items[1:-1])
    
def check_if_new_transaction(items):
    global date
    global balance
    global details
    
    last_item_is_number = is_float(items[-1])
    details_exist = details != ""
    starts_with_date = is_date(items[0])
    
    if last_item_is_number:
        if details_exist:
            if starts_with_date:
                enter_row_into_statement_dict(date,details,amount,balance)
                date = items[0]  
                check_if_two_transactions(items)
            
            else:
                enter_row_into_statement_dict(date, details, amount, "")
                check_if_two_transactions(items)

        else:
            date = items[0]
            details = " ".join(items[1:-1])
            balance = items[-1].replace(",","")
    else:
        details = details + " " + " ".join(items)
    
def transform_statement_to_df(statement):
    for lines in statement:
        row_items = lines.split()
        check_if_new_transaction(row_items)
        
    enter_row_into_statement_dict(date,details,"",balance)
    
    df = pd.DataFrame(statement_dict, columns= ["Date", "Transaction Details", "Amount", "Balance"])
    
    return df

In [5]:
def check_statement_for_dates(statement):
    global year
    
    first_date = statement[0].split()[0]
    last_date = statement[-1].split()[0]

    if not(is_date(first_date) & is_date(last_date)):
        print("Statement Dates not found")
        print(first_date)
    else:
        print("Statement date found = ", last_date, year, "\nPrev statement date = ", first_date, year)

def get_statement(invoice, text):
    global year
    
    year = invoice[invoice.find('/')+1:invoice.find('-')]
    start = [i for i, s in enumerate(text.split("\n")) if 'B/F BALANCE' in s]
    end = [i for i, s in enumerate(text.split("\n")) if 'C/F BALANCE' in s]

    statement = [row.replace(",", "") for row in text.split("\n")[start[0]:end[0]+1]]
    check_statement_for_dates(statement)
    return statement

In [6]:
def convert(invoice):
    with pdfplumber.open(invoice) as pdf:
        page = pdf.pages[0]
        text = page.extract_text(x_tolerance=1)
        
    statement = get_statement(invoice, text)
    return transform_statement_to_df(statement)

## Global Variables

In [7]:
year = ""

In [8]:
statement_dict = {
    "Date": [],
    "Transaction Details": [], 
    "Amount": [],
    "Balance": [] 
}

In [9]:
date = ""
balance = ""
amount = ""
details = ""

# ==========================================

## START HERE

In [10]:
invoice = 'statements/2020-08.pdf'

In [11]:
convert(invoice)

Statement date found =  12-Aug 2020 
Prev statement date =  11-Jul 2020


,Date,Transaction Details,Amount,Balance
0,11-Jul,2020 B/F BALANCE,,77817.48
1,20-Jul,TO PAYME(HSBC) T2007188F952(18JUL20),90.00,
2,20-Jul,PAYME(HSBC) T200720A7536(20JUL20),560.00,77167.48
3,23-Jul,FROM WEARNE JEREMY R N72399865235(23JUL20),1680.00,78847.48
4,24-Jul,CHAN J**** HC12072460325323 24JUL,400.00,
5,24-Jul,PAYME(HSBC) T200724GP139(24JUL20),560.00,77887.48
6,28-Jul,CREDIT CARD PAYMENT 4966040127767358,838.10,
7,28-Jul,INTEREST,0.06,77049.44
8,31-Jul,CHAN J**** HC12073163993770 31JUL,137.00,76912.44
9,1-Aug,TO PAYME(HSBC) T200801TG146(01AUG20),560.00,


# ==========================================

## Clean Up

In [12]:
clear_statement_dict()
statement_dict

{'Date': [], 'Transaction Details': [], 'Amount': [], 'Balance': []}